In [10]:
# packages
import gzip
import numpy as np
import pandas as pd
import json
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import linear_model

In [2]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

In [ ]:

review_filepath = "/Users/paigepagaduan/Desktop/cse158/data/review-Idaho.json.gz"
meta_filepath = "/Users/paigepagaduan/Desktop/cse158/data/meta-Idaho.json.gz"

reviews = []
bizs = []
for l in parse(review_filepath):
    reviews.append(l)

for l in parse(meta_filepath):
    bizs.append(l)

In [4]:
# from eda notebook

b_avg_ratings = []
b_num_reviews = []
b_num_similar_bizs = []
b_prices = []
b_states = [] # i.e. None, 'Closed ⋅ Opens 10AM Fri' , 'Permanently closed' , 'Open ⋅ Closes 5PM' , maybe more types
# ^ only listed once per biz, not at time of each review meaning the open/closed is not always true for every one of it's reviews, but maybe 'permanently closed' would help predict time
b_has_prices = []
for b in bizs:
    b_avg_ratings.append(b['avg_rating'])
    b_num_reviews.append(b['num_of_reviews'])
    b_num_similar_bizs.append((0 if b['relative_results'] == None else len(b['relative_results'])))
    if b['price'] == None:
        b_has_prices.append(0)
        b_prices.append(b['price']) # missing data
    else:
        b_has_prices.append(1)
        b_prices.append(len(b['price']))
    try:
        b_states.append(b['state']) # maybe split into counts of each one when analyzing
    except Exception as e:
        b_states.append(None)
r_ratings = []
r_time_difs = []
r_num_pics = []
r_has_response = []
r_resp_times = []
for r in reviews:
    r_ratings.append(r['rating'])
    try:
        r_time_difs.append(r['resp']['time'] - r['time']) #Nones might cause errors or NaNs, handle later
        r_resp_times.append(r['resp']['time'])
        r_has_response.append(1)
    except Exception as e:
        r_time_difs.append(None)
        r_resp_times.append(None)
        r_has_response.append(0)
    try:
        r_num_pics.append(len(r['pics']))
    except Exception as e:
        r_num_pics.append(0)

In [5]:
reviews_with_responses = [review for review in reviews if review.get("resp")]


In [6]:
for r in reviews_with_responses:
    r['resp_time'] = r['resp']['time'] - r['time']

In [7]:
reviews_with_responses[0]

{'user_id': '115114032166130224762',
 'name': 'Mariah Schaeffer',
 'time': 1589989743506,
 'rating': 5,
 'text': 'Kevin and Shannon are amazing! They were very sweet and made the job fast! Also they are one of the cheeper junk removal services! I will for sure use again!',
 'pics': None,
 'resp': {'time': 1591139557205,
  'text': 'Thank you for hiring us for your junk removal!'},
 'gmap_id': '0x54afb4c19c4bffff:0x9389114191ca2781',
 'resp_time': 1149813699}

# Model #1 - linear regression

In [ ]:
def feature(d):
    feat = [1]
    try:
        text_length = len(d['text'])
    except (KeyError, TypeError):
        text_length = 0 
    feat.append(text_length)
    feat.append(d.get('rating', 0))
    feat.append(d.get('time', 0))
    return feat

In [12]:

X = [feature(r) for r in reviews_with_responses]
y = [r['resp_time'] for r in reviews_with_responses]


In [13]:
model = sklearn.linear_model.LinearRegression(fit_intercept=False)
model.fit(X, y)

LinearRegression(fit_intercept=False)

In [14]:
y_pred = model.predict(X)
sse = sum([x**2 for x in (y - y_pred)])
mse = sse / len(y)
mse

1.656602929714003e+20